In [2]:
## parse the gene annotation to get G gene and F gene
from Bio import SeqIO
import os,sys
import pandas as pd
import phylopandas as ph

In [3]:
### parse the annotatio information
ano = pd.read_csv('HRSV_0622_Annotations.csv')
CDS = ano.loc[ano['Type'] == 'CDS']
CDS_name = CDS.groupby(['Name']).count()
CDS_name.to_csv("HRSV_annotation_CDS.csv")

In [8]:
def getgene(fasta_file, anno_list):
    f = open(fasta_file, "w")
    for rec in SeqIO.parse("HRSV_0622.gb", "gb"):
        for feature in rec.features:
            for key, val in feature.qualifiers.items():
                if feature.type == "CDS":
                    if any (s in val for s in anno_list):
                        print (">" + rec.id, file=f )
                        print (feature.location.extract(rec).seq,file=f)
    f.close()

In [11]:
def duplicate_remover(fasta_file):
    df = ph.read_fasta(fasta_file)
    df = df.filter(['id','sequence'], axis=1)
    df = df.drop_duplicates()
    df.to_csv("temp.tab", sep="\t",index = False,header=False)
    df2= SeqIO.parse("temp.tab", "tab")
    SeqIO.write(df2, "rm"+fasta_file, "fasta")
    os.remove("temp.tab")
    print("duplicate records removed!\n"+ str(len(df.index))+" unique records saved from " + fasta_file)

In [4]:
def sequence_cleaner(fasta_file, min_length=0, por_n=100):
    output_file = open("clear_" + fasta_file, "w")

    for seq_record in SeqIO.parse(fasta_file, "fasta"):
        if (len(seq_record.seq) >= min_length and (float((seq_record.seq).count("N"))/float(len(seq_record.seq)))*100 <= por_n):
            output_file.write(">" + seq_record.id + "\n" + str(seq_record.seq) + "\n")

    print("CLEAN!!!\nPlease check clear_" + fasta_file)

In [9]:
F={"F",
"F",
"f ",
"F glycoprotein ",
"F protein ",
"firefly luciferase ",
"fusion",
"fusion glycoprotein",
"fusion protein",
"fusion protein F",
"fusion protein precursor",
"truncated fusion protein" 
    
}

In [10]:

getgene("HRSV_F.fasta",F)

In [12]:
duplicate_remover("HRSV_F.fasta")

duplicate records removed!
5183 unique records saved from HRSV_F.fasta


In [5]:
## length of HRSV F : 2155 bp:
sequence_cleaner("rmHRSV_F.fasta",2155*0.7,20)

CLEAN!!!
Please check clear_rmHRSV_F.fasta


In [ ]:
## remove the recombinant and artificial constructed strain with sequence id


In [ ]:
## get taxa for F gene
